In [1]:
import pandas as pd
import numpy as np
import seaborn as sns                       #visualisation
import matplotlib.pyplot as plt             #visualisation    
sns.set(color_codes=True)

pd.set_option('display.max_columns', None)

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


In [2]:
# Load the dataset
data_path = '/Users/kajalshukla/Desktop/MLOPs/Assignments/Assignment-1/adsp_ml_ops/data/athletes.csv'
data = pd.read_csv(data_path)

In [3]:
data.head(3)

,age,height,weight,candj,snatch,deadlift,backsq,total_lift,region_Asia,region_Australia,region_Canada East,region_Canada West,region_Central East,region_Europe,region_Latin America,region_Mid Atlantic,region_North Central,region_North East,region_North West,region_Northern California,region_South Central,region_South East,region_South West,region_Southern California,gender_Female,gender_Male,background_I have no athletic background besides CrossFit|Decline to answer|,background_I have no athletic background besides CrossFit|I played college sports|,background_I have no athletic background besides CrossFit|I played college sports|I played professional sports|,background_I have no athletic background besides CrossFit|I played college sports|I regularly play recreational sports|,background_I have no athletic background besides CrossFit|I played professional sports|,background_I have no athletic background besides CrossFit|I played professional sports|I regularly play recreational sports|,background_I have no athletic background besides CrossFit|I played youth or high school level sports|,background_I have no athletic background besides CrossFit|I played youth or high school level sports|I played college sports|,background_I have no athletic background besides CrossFit|I played youth or high school level sports|I played college sports|I played professional sports|,background_I have no athletic background besides CrossFit|I played youth or high school level sports|I played college sports|I played professional sports|I regularly play recreational sports|Decline to answer|,background_I have no athletic background besides CrossFit|I played youth or high school level sports|I played college sports|I regularly play recreational sports|,background_I have no athletic background besides CrossFit|I played youth or high school level sports|I played professional sports|,background_I have no athletic background besides CrossFit|I played youth or high school level sports|I played professional sports|I regularly play recreational sports|,background_I have no athletic background besides CrossFit|I played youth or high school level sports|I regularly play recreational sports|,background_I have no athletic background besides CrossFit|I regularly play recreational sports|,background_I played college sports|,background_I played college sports|Decline to answer|,background_I played college sports|I played professional sports|,background_I played college sports|I played professional sports|I regularly play recreational sports|,background_I played college sports|I regularly play recreational sports|,background_I played professional sports|,background_I played professional sports|I regularly play recreational sports|,background_I played youth or high school level sports|,background_I played youth or high school level sports|Decline to answer|,background_I played youth or high school level sports|I played college sports|,background_I played youth or high school level sports|I played college sports|I played professional sports|,background_I played youth or high school level sports|I played college sports|I played professional sports|I regularly play recreational sports|,background_I played youth or high school level sports|I played college sports|I regularly play recreational sports|,background_I played youth or high school level sports|I played professional sports|,background_I played youth or high school level sports|I played professional sports|I regularly play recreational sports|,background_I played youth or high school level sports|I regularly play recreational sports|,background_I played youth or high school level sports|I regularly play recreational sports|Decline to answer|,background_I regularly play recreational sports|,background_I regularly play recreational sports|Decline to answer|,experience_I began CrossFit by trying it alone (without a coach)|Decline to answer|,experience_I began CrossFit by trying it alone (without a coach)|I began Cross

# Raw data

## Data Cleaning

In [4]:
## Remove null values of relevant columns
relevant_columns = ['region', 'age', 'weight', 'height', 'howlong', 'gender', 'eat', 'train', 
                    'background', 'experience', 'schedule', 'deadlift', 'candj', 'snatch', 'backsq']
data = data.dropna(subset=relevant_columns)

KeyError: ['region', 'howlong', 'gender', 'eat', 'train', 'background', 'experience', 'schedule']

In [57]:
## Remove irrelevant columns
irrelevant_columns = ['affiliate', 'team', 'name', 'athlete_id', 'fran', 'helen', 'grace',
                      'filthy50', 'fgonebad', 'run400', 'run5k', 'pullups', 'train']
data = data.drop(columns=irrelevant_columns)

In [58]:
data.head(5)

,region,gender,age,height,weight,candj,snatch,deadlift,backsq,eat,background,experience,schedule,howlong
6,South Central,Male,21.0,72.0,175.0,0.0,0.0,0.0,0.0,I eat quality foods but don't measure the amou...,I have no athletic background besides CrossFit|,I began CrossFit with a coach (e.g. at an affi...,I do multiple workouts in a day 3+ times a wee...,2-4 years|
13,Central East,Male,43.0,71.0,185.0,0.0,0.0,0.0,0.0,I eat quality foods but don't measure the amount|,I have no athletic background besides CrossFit|,I began CrossFit with a coach (e.g. at an affi...,I usually only do 1 workout a day|I typically ...,2-4 years|
21,Southern California,Male,30.0,71.0,200.0,235.0,175.0,385.0,315.0,I eat whatever is convenient|,I played youth or high school level sports|I p...,I began CrossFit by trying it alone (without a...,I do multiple workouts in a day 1x a week|I ty...,1-2 years|
22,Africa,Male,28.0,70.0,176.0,187.0,134.0,335.0,254.0,I eat 1-3 full cheat meals per week|,I have no athletic background besides CrossFit|,I began CrossFit with a coach (e.g. at an affi...,I do multiple workouts in a day 1x a week|,2-4 years|
27,North East,Male,35.0,68.0,225.0,285.0,205.0,440.0,405.0,I eat quality foods but don't measure the amount|,I played youth or high school level sports|,I began CrossFit with a coach (e.g. at an affi...,I typically rest 4 or more days per month|,2-4 years|


In [59]:
# Clean survey data
decline_dict = {'Decline to answer|': np.nan}
data = data.replace(decline_dict)
data = data.dropna(subset=['background', 'experience', 'schedule', 'howlong', 'eat'])


### Determine the highest variability in columns to calculate total_lift

In [60]:
#Calculate standard deviation
numeric_columns = ['age', 'weight', 'height', 'howlong', 'deadlift', 'candj', 'snatch', 'backsq']

data_v1_std_devs = data[numeric_columns].std()

#Sort columns by their standard devation
sorted_columns_v1 = data_v1_std_devs.sort_values(ascending=False).index.tolist()

/var/folders/dn/9qflv0c57rl7ynhwt9tnqmrr0000gn/T/ipykernel_4442/2011793858.py:4: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  data_v1_std_devs = data[numeric_columns].std()


In [61]:
print(sorted_columns_v1)

['deadlift', 'height', 'backsq', 'snatch', 'candj', 'weight', 'age']


### The highest variability to calculate total_lift is 'deadlift', 'backsq', 'candj', 'snatch'

In [68]:
data['total_lift'] = data['deadlift'] + data['candj'] + data['snatch'] + data['backsq']


In [69]:
data = data.drop_duplicates()
print(data.isnull().sum())

region        0
gender        0
age           0
height        0
weight        0
candj         0
snatch        0
deadlift      0
backsq        0
eat           0
background    0
experience    0
schedule      0
howlong       0
total_lift    0
dtype: int64


## Dealing with Categorical Values

In [70]:
# One-Hot Encoding for categorical columns
categorical_cols = ['region', 'gender', 'eat', 'background', 'experience', 'schedule', 'howlong']
data = pd.get_dummies(data, columns=categorical_cols, drop_first=True)

In [71]:
# Convert float columns to int
float_cols = ['age', 'height', 'weight', 'candj', 'snatch', 'deadlift', 'backsq', 'total_lift']
for col in float_cols:
    data[col] = data[col].astype(int)

In [72]:
# Verify the changes
pd.set_option('display.max_rows', None)  # Display all rows
pd.set_option('display.max_columns', None)  # Display all columns

data.dtypes

age                                                                                                                                                                                                                                                                                                                           int64
height                                                                                                                                                                                                                                                                                                                        int64
weight                                                                                                                                                                                                                                                                                                                        int64
candj                       

In [73]:
#Split data for data_v1
X_v1  = data.drop('total_lift', axis=1)
y_v1 = data['total_lift']
X_train_v1, X_test_v1, y_train_v1, y_test_v1 = train_test_split(X_v1, y_v1, test_size=0.2, random_state=42) 


## Save the raw data and push it using dvc tool

In [74]:
pip install dvc

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [75]:
!dvc init


Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


In [76]:
!git status

On branch monika_v2
Your branch is ahead of 'origin/monika_v2' by 4 commits.
  (use "git push" to publish your local commits)

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	modified:   .dvc/config

Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   data.ipynb
	deleted:    versions/.gitignore
	deleted:    versions/data_v1.csv
	deleted:    versions/data_v1.csv.dvc
	deleted:    versions/data_v2.csv.dvc



In [77]:
!git add .

In [78]:
!git commit -m "Initialize DVC"

[monika_v2 4256681] Initialize DVC
 6 files changed, 5110 insertions(+), 32197 deletions(-)
 delete mode 100644 versions/.gitignore
 delete mode 100644 versions/data_v1.csv
 delete mode 100644 versions/data_v1.csv.dvc
 delete mode 100644 versions/data_v2.csv.dvc


In [79]:
data.to_csv('data/athletes.csv', index=False)


In [81]:
!git rm --cached data/athletes.csv

rm 'data/athletes.csv'


In [82]:
!git commit -m "Stop tracking data/athletes.csv with Git"

[monika_v2 4c23e5c] Stop tracking data/athletes.csv with Git
 1 file changed, 423007 deletions(-)
 delete mode 100644 data/athletes.csv


In [83]:
!dvc add data/athletes.csv


⠋ Checking graph                                       core>
Adding...                                                                       
!
                                                                                
!
  0% Checking cache in '/Users/kajalshukla/Desktop/MLOPs/Assignments/Assignment-
                                                                                
!
  0%|          |Adding data/athletes.csv to cache     0/? [00:00<?,     ?file/s]
  0%|          |Adding data/athletes.csv to cache     0/1 [00:00<?,     ?file/s]
  0%|          |Adding data/athletes.csv to cache     0/1 [00:00<?,     ?file/s]
                                                                                
!
  0%|          |Checking out /Users/kajalshukla/Deskto0/? [00:00<?,    ?files/s]
  0%|          |Checking out /Users/kajalshukla/Deskto0/1 [00:00<?,    ?files/s]
100% Adding...|████████████████████████████████████████|1/1 [00:00, 28.94file/s]

To track the changes with git, run:

	g

In [84]:
!git add data/athletes.csv.dvc .gitignore


In [85]:
!git commit -m "Track data/athletes.csv with DVC"

[monika_v2 0d2bde5] Track data/athletes.csv with DVC
 1 file changed, 5 insertions(+)
 create mode 100644 data/athletes.csv.dvc


## Set up remote storage with DVC

In [86]:
! pip install dvc[gdrive]


zsh:1: no matches found: dvc[gdrive]


In [87]:
!dvc remote add -d myremote gdrive://1Zpt1VhpAx-XdTvY5W2Yhzs7-24SS23YQ


Setting 'myremote' as a default remote.


In [ ]:
!dvc commit data/athletes.csv.dvc

In [88]:
!dvc push


  0% Pushing to gdrive://1Zpt1VhpAx-XdTvY5W2Yhzs7-24SS23YQ/files/md5| |0/1 [00:0
!
  0%|          |/Users/kajalshukla/Desktop/MLOPs/As0.00/? [00:00<?,        ?B/s]
  0%|          |/Users/kajalshukla/Desktop/MLOP0.00/19.0M [00:00<?,        ?B/s]
  0%|          |/Users/kajalshukla/Deskt8.00k/19.0M [00:02<1:50:36,    3.01kB/s]
  4%|▍         |/Users/kajalshukla/Desktop/776k/19.0M [00:02<00:48,     395kB/s]
 35%|███▌      |/Users/kajalshukla/Desktop6.69M/19.0M [00:02<00:02,    4.49MB/s]
100% Pushing to gdrive://1Zpt1VhpAx-XdTvY5W2Yhzs7-24SS23YQ/files/md5|█|1/1 [00:0
1 file pushed                                                                   


In [89]:
!git commit data/athletes.csv.dvc -m "raw data"


Fetching
!
  0% Checking cache in '/Users/kajalshukla/Desktop/MLOPs/Assignments/Assignment-
Fetching                                                                        
Building workspace index                              |2.00 [00:00,  499entry/s]
Comparing indexes                                     |3.00 [00:00,  922entry/s]
Applying changes                                      |0.00 [00:00,     ?file/s]
Everything is up to date.


In [90]:
!git add .dvc/config


In [ ]:
!git push

# Version - 2

## Remove Outliers

In [92]:
data_v2 = data

In [93]:
data_v2.head(2)

,age,height,weight,candj,snatch,deadlift,backsq,total_lift,region_Asia,region_Australia,region_Canada East,region_Canada West,region_Central East,region_Europe,region_Latin America,region_Mid Atlantic,region_North Central,region_North East,region_North West,region_Northern California,region_South Central,region_South East,region_South West,region_Southern California,gender_Female,gender_Male,eat_I eat 1-3 full cheat meals per week|Decline to answer|,eat_I eat quality foods but don't measure the amount|,eat_I eat quality foods but don't measure the amount|Decline to answer|,eat_I eat quality foods but don't measure the amount|I eat 1-3 full cheat meals per week|,eat_I eat quality foods but don't measure the amount|I eat 1-3 full cheat meals per week|Decline to answer|,eat_I eat quality foods but don't measure the amount|I eat whatever is convenient|,eat_I eat quality foods but don't measure the amount|I eat whatever is convenient|I eat 1-3 full cheat meals per week|,eat_I eat quality foods but don't measure the amount|I eat whatever is convenient|I eat 1-3 full cheat meals per week|Decline to answer|,eat_I eat strict Paleo|,eat_I eat strict Paleo|I eat 1-3 full cheat meals per week|,eat_I eat strict Paleo|I eat quality foods but don't measure the amount|,eat_I eat strict Paleo|I eat quality foods but don't measure the amount|I eat 1-3 full cheat meals per week|,eat_I eat strict Paleo|I eat quality foods but don't measure the amount|I eat whatever is convenient|,eat_I eat strict Paleo|I eat quality foods but don't measure the amount|I eat whatever is convenient|I eat 1-3 full cheat meals per week|,eat_I eat strict Paleo|I eat whatever is convenient|,eat_I eat strict Paleo|I eat whatever is convenient|I eat 1-3 full cheat meals per week|,eat_I eat whatever is convenient|,eat_I eat whatever is convenient|Decline to answer|,eat_I eat whatever is convenient|I eat 1-3 full cheat meals per week|,eat_I eat whatever is convenient|I eat 1-3 full cheat meals per week|Decline to answer|,eat_I weigh and measure my food|,eat_I weigh and measure my food|Decline to answer|,eat_I weigh and measure my food|I eat 1-3 full cheat meals per week|,eat_I weigh and measure my food|I eat 1-3 full cheat meals per week|Decline to answer|,eat_I weigh and measure my food|I eat quality foods but don't measure the amount|,eat_I weigh and measure my food|I eat quality foods but don't measure the amount|I eat 1-3 full cheat meals per week|,eat_I weigh and measure my food|I eat quality foods but don't measure the amount|I eat whatever is convenient|,eat_I weigh and measure my food|I eat quality foods but don't measure the amount|I eat whatever is convenient|I eat 1-3 full cheat meals per week|,eat_I weigh and measure my food|I eat strict Paleo|,eat_I weigh and measure my food|I eat strict Paleo|I eat 1-3 full cheat meals per week|,eat_I weigh and measure my food|I eat strict Paleo|I eat quality foods but don't measure the amount|,eat_I weigh and measure my food|I eat strict Paleo|I eat quality foods but don't measure the amount|I eat 1-3 full cheat meals per week|,eat_I weigh and measure my food|I eat strict Paleo|I eat quality foods but don't measure the amount|I eat 1-3 full cheat meals per week|Decline to answer|,eat_I weigh and measure my food|I eat strict Paleo|I eat quality foods but don't measure the amount|I eat whatever is convenient|I eat 1-3 full cheat meals per week|,eat_I weigh and measure my food|I eat strict Paleo|I eat quality foods but don't measure the amount|I eat whatever is convenient|I eat 1-3 full cheat meals per week|Decline to answer|,eat_I weigh and measure my food|I eat strict Paleo|I eat whatever is convenient|,eat_I weigh and measure my food|I eat strict Paleo|I eat whatever is convenient|I eat 1-3 full cheat meals per week|,eat_I weigh and measure my food|I eat whatever is convenient|,eat_I weigh and measure my food|I eat whatever is convenient|I eat 1-3 full cheat meals per week|,background_I have no athletic background besides Cro

In [94]:
# Remove outliers
data_v2 = data_v2[data_v2['weight'] < 1500]
# data = data[data['gender'] != '--']
data_v2 = data_v2[data_v2['age'] >= 18]
data_v2 = data_v2[(data_v2['height'] < 96) & (data_v2['height'] > 48)]

data_v2 = data_v2[
    (data_v2['deadlift'] > 0) & (data_v2['deadlift'] <= 1105) | 
    ((data_v2['gender_Female'] == 1) & (data_v2['deadlift'] <= 636))
]
data_v2 = data_v2[(data_v2['candj'] > 0) & (data_v2['candj'] <= 395)]
data_v2 = data_v2[(data_v2['snatch'] > 0) & (data_v2['snatch'] <= 496)]
data_v2 = data_v2[(data_v2['backsq'] > 0) & (data_v2['backsq'] <= 1069)]

In [95]:
data_v2.head(2)

,age,height,weight,candj,snatch,deadlift,backsq,total_lift,region_Asia,region_Australia,region_Canada East,region_Canada West,region_Central East,region_Europe,region_Latin America,region_Mid Atlantic,region_North Central,region_North East,region_North West,region_Northern California,region_South Central,region_South East,region_South West,region_Southern California,gender_Female,gender_Male,eat_I eat 1-3 full cheat meals per week|Decline to answer|,eat_I eat quality foods but don't measure the amount|,eat_I eat quality foods but don't measure the amount|Decline to answer|,eat_I eat quality foods but don't measure the amount|I eat 1-3 full cheat meals per week|,eat_I eat quality foods but don't measure the amount|I eat 1-3 full cheat meals per week|Decline to answer|,eat_I eat quality foods but don't measure the amount|I eat whatever is convenient|,eat_I eat quality foods but don't measure the amount|I eat whatever is convenient|I eat 1-3 full cheat meals per week|,eat_I eat quality foods but don't measure the amount|I eat whatever is convenient|I eat 1-3 full cheat meals per week|Decline to answer|,eat_I eat strict Paleo|,eat_I eat strict Paleo|I eat 1-3 full cheat meals per week|,eat_I eat strict Paleo|I eat quality foods but don't measure the amount|,eat_I eat strict Paleo|I eat quality foods but don't measure the amount|I eat 1-3 full cheat meals per week|,eat_I eat strict Paleo|I eat quality foods but don't measure the amount|I eat whatever is convenient|,eat_I eat strict Paleo|I eat quality foods but don't measure the amount|I eat whatever is convenient|I eat 1-3 full cheat meals per week|,eat_I eat strict Paleo|I eat whatever is convenient|,eat_I eat strict Paleo|I eat whatever is convenient|I eat 1-3 full cheat meals per week|,eat_I eat whatever is convenient|,eat_I eat whatever is convenient|Decline to answer|,eat_I eat whatever is convenient|I eat 1-3 full cheat meals per week|,eat_I eat whatever is convenient|I eat 1-3 full cheat meals per week|Decline to answer|,eat_I weigh and measure my food|,eat_I weigh and measure my food|Decline to answer|,eat_I weigh and measure my food|I eat 1-3 full cheat meals per week|,eat_I weigh and measure my food|I eat 1-3 full cheat meals per week|Decline to answer|,eat_I weigh and measure my food|I eat quality foods but don't measure the amount|,eat_I weigh and measure my food|I eat quality foods but don't measure the amount|I eat 1-3 full cheat meals per week|,eat_I weigh and measure my food|I eat quality foods but don't measure the amount|I eat whatever is convenient|,eat_I weigh and measure my food|I eat quality foods but don't measure the amount|I eat whatever is convenient|I eat 1-3 full cheat meals per week|,eat_I weigh and measure my food|I eat strict Paleo|,eat_I weigh and measure my food|I eat strict Paleo|I eat 1-3 full cheat meals per week|,eat_I weigh and measure my food|I eat strict Paleo|I eat quality foods but don't measure the amount|,eat_I weigh and measure my food|I eat strict Paleo|I eat quality foods but don't measure the amount|I eat 1-3 full cheat meals per week|,eat_I weigh and measure my food|I eat strict Paleo|I eat quality foods but don't measure the amount|I eat 1-3 full cheat meals per week|Decline to answer|,eat_I weigh and measure my food|I eat strict Paleo|I eat quality foods but don't measure the amount|I eat whatever is convenient|I eat 1-3 full cheat meals per week|,eat_I weigh and measure my food|I eat strict Paleo|I eat quality foods but don't measure the amount|I eat whatever is convenient|I eat 1-3 full cheat meals per week|Decline to answer|,eat_I weigh and measure my food|I eat strict Paleo|I eat whatever is convenient|,eat_I weigh and measure my food|I eat strict Paleo|I eat whatever is convenient|I eat 1-3 full cheat meals per week|,eat_I weigh and measure my food|I eat whatever is convenient|,eat_I weigh and measure my food|I eat whatever is convenient|I eat 1-3 full cheat meals per week|,background_I have no athletic background besides Cro

## Feature Engineering

### Body Mass Index

In [96]:
data_v2['bmi'] = data_v2['weight'] / (data_v2['height'] / 100)**2

In [97]:
data_v2 = data_v2.drop(columns=["eat_I eat 1-3 full cheat meals per week|Decline to answer|",
                          "eat_I eat quality foods but don't measure the amount|",
                          "eat_I eat quality foods but don't measure the amount|Decline to answer|",
                          "eat_I eat quality foods but don't measure the amount|I eat 1-3 full cheat meals per week|",
                          "eat_I eat quality foods but don't measure the amount|I eat 1-3 full cheat meals per week|Decline to answer|",
                          "eat_I eat quality foods but don't measure the amount|I eat whatever is convenient|",
                          "eat_I eat quality foods but don't measure the amount|I eat whatever is convenient|I eat 1-3 full cheat meals per week|",
                          "eat_I eat quality foods but don't measure the amount|I eat whatever is convenient|I eat 1-3 full cheat meals per week|Decline to answer|",
                          "eat_I eat strict Paleo|",
                          "eat_I eat strict Paleo|I eat 1-3 full cheat meals per week|",
                          "eat_I eat strict Paleo|I eat quality foods but don't measure the amount|",
                          "eat_I eat strict Paleo|I eat quality foods but don't measure the amount|I eat 1-3 full cheat meals per week|",
                          "eat_I eat strict Paleo|I eat quality foods but don't measure the amount|I eat whatever is convenient|",
                          "eat_I eat strict Paleo|I eat quality foods but don't measure the amount|I eat whatever is convenient|I eat 1-3 full cheat meals per week|",
                          "eat_I eat strict Paleo|I eat whatever is convenient|",
                          "eat_I eat strict Paleo|I eat whatever is convenient|I eat 1-3 full cheat meals per week|",
                          "eat_I eat whatever is convenient|",
                          "eat_I eat whatever is convenient|Decline to answer|",
                          "eat_I eat whatever is convenient|I eat 1-3 full cheat meals per week|",
                          "eat_I eat whatever is convenient|I eat 1-3 full cheat meals per week|Decline to answer|",
                          "eat_I weigh and measure my food|",
                          "eat_I weigh and measure my food|Decline to answer|",
                          "eat_I weigh and measure my food|I eat 1-3 full cheat meals per week|",
                          "eat_I weigh and measure my food|I eat 1-3 full cheat meals per week|Decline to answer|",
                          "eat_I weigh and measure my food|I eat quality foods but don't measure the amount|",
                          "eat_I weigh and measure my food|I eat quality foods but don't measure the amount|I eat 1-3 full cheat meals per week|",                                                                                                                                                                                                         
                          "eat_I weigh and measure my food|I eat quality foods but don't measure the amount|I eat whatever is convenient|",                                                                                                                                                                                                                
                          "eat_I weigh and measure my food|I eat quality foods but don't measure the amount|I eat whatever is convenient|I eat 1-3 full cheat meals per week|",                                                                                                                                                                            
                          "eat_I weigh and measure my food|I eat strict Paleo|",                                                                                                                                                                                                                                                                           
                          "eat_I weigh and measure my food|I eat strict Paleo|I eat 1-3 full cheat meals per week|",                                                                                                                                                                                                                                       
                          "eat_I weigh and measure my food|I eat strict Paleo|I eat quality foods but don't measure the amount|",                                                                                                                                                                                                                          
                          "eat_I weigh and measure my food|I eat strict Paleo|I eat quality foods but don't measure the amount|I eat 1-3 full cheat meals per week|",                                                                                                                                                                                      
                          "eat_I weigh and measure my food|I eat strict Paleo|I eat quality foods but don't measure the amount|I eat 1-3 full cheat meals per week|Decline to answer|",                                                                                                                                                                    
                          "eat_I weigh and measure my food|I eat strict Paleo|I eat quality foods but don't measure the amount|I eat whatever is convenient|I eat 1-3 full cheat meals per week|",                                                                                                                                                         
                          "eat_I weigh and measure my food|I eat strict Paleo|I eat quality foods but don't measure the amount|I eat whatever is convenient|I eat 1-3 full cheat meals per week|Decline to answer|",                                                                                                                                       
                          "eat_I weigh and measure my food|I eat strict Paleo|I eat whatever is convenient|",                                                                                                                                                                                                                                              
                          "eat_I weigh and measure my food|I eat strict Paleo|I eat whatever is convenient|I eat 1-3 full cheat meals per week|",                                                                                                                                                                                                          
                          "eat_I weigh and measure my food|I eat whatever is convenient|",                                                                                                                                                                                                                                                                 
                          "eat_I weigh and measure my food|I eat whatever is convenient|I eat 1-3 full cheat meals per week|"
                            ])

In [98]:
data_v2.head(3)

,age,height,weight,candj,snatch,deadlift,backsq,total_lift,region_Asia,region_Australia,region_Canada East,region_Canada West,region_Central East,region_Europe,region_Latin America,region_Mid Atlantic,region_North Central,region_North East,region_North West,region_Northern California,region_South Central,region_South East,region_South West,region_Southern California,gender_Female,gender_Male,background_I have no athletic background besides CrossFit|Decline to answer|,background_I have no athletic background besides CrossFit|I played college sports|,background_I have no athletic background besides CrossFit|I played college sports|I played professional sports|,background_I have no athletic background besides CrossFit|I played college sports|I regularly play recreational sports|,background_I have no athletic background besides CrossFit|I played professional sports|,background_I have no athletic background besides CrossFit|I played professional sports|I regularly play recreational sports|,background_I have no athletic background besides CrossFit|I played youth or high school level sports|,background_I have no athletic background besides CrossFit|I played youth or high school level sports|I played college sports|,background_I have no athletic background besides CrossFit|I played youth or high school level sports|I played college sports|I played professional sports|,background_I have no athletic background besides CrossFit|I played youth or high school level sports|I played college sports|I played professional sports|I regularly play recreational sports|Decline to answer|,background_I have no athletic background besides CrossFit|I played youth or high school level sports|I played college sports|I regularly play recreational sports|,background_I have no athletic background besides CrossFit|I played youth or high school level sports|I played professional sports|,background_I have no athletic background besides CrossFit|I played youth or high school level sports|I played professional sports|I regularly play recreational sports|,background_I have no athletic background besides CrossFit|I played youth or high school level sports|I regularly play recreational sports|,background_I have no athletic background besides CrossFit|I regularly play recreational sports|,background_I played college sports|,background_I played college sports|Decline to answer|,background_I played college sports|I played professional sports|,background_I played college sports|I played professional sports|I regularly play recreational sports|,background_I played college sports|I regularly play recreational sports|,background_I played professional sports|,background_I played professional sports|I regularly play recreational sports|,background_I played youth or high school level sports|,background_I played youth or high school level sports|Decline to answer|,background_I played youth or high school level sports|I played college sports|,background_I played youth or high school level sports|I played college sports|I played professional sports|,background_I played youth or high school level sports|I played college sports|I played professional sports|I regularly play recreational sports|,background_I played youth or high school level sports|I played college sports|I regularly play recreational sports|,background_I played youth or high school level sports|I played professional sports|,background_I played youth or high school level sports|I played professional sports|I regularly play recreational sports|,background_I played youth or high school level sports|I regularly play recreational sports|,background_I played youth or high school level sports|I regularly play recreational sports|Decline to answer|,background_I regularly play recreational sports|,background_I regularly play recreational sports|Decline to answer|,experience_I began CrossFit by trying it alone (without a coach)|Decline to answer|,experience_I began CrossFit by trying it alone (without a coach)|I began Cross

## Save version 2

In [99]:
data_v2.to_csv('data/athletes.csv', index=False)

In [100]:
!dvc add data/athletes.csv

⠋ Checking graph                                       core>
Adding...                                                                       
!
                                                                                
!
  0% Checking cache in '/Users/kajalshukla/Desktop/MLOPs/Assignments/Assignment-
                                                                                
!
  0%|          |Adding data/athletes.csv to cache     0/? [00:00<?,     ?file/s]
  0%|          |Adding data/athletes.csv to cache     0/1 [00:00<?,     ?file/s]
  0%|          |Adding data/athletes.csv to cache     0/1 [00:00<?,     ?file/s]
                                                                                
!
  0%|          |Checking out /Users/kajalshukla/Deskto0/? [00:00<?,    ?files/s]
  0%|          |Checking out /Users/kajalshukla/Deskto0/1 [00:00<?,    ?files/s]
100% Adding...|████████████████████████████████████████|1/1 [00:00, 38.66file/s]

To track the changes with git, run:

	g

In [ ]:
!dvc commit data/athletes.csv.dvc

In [ ]:
!dvc push

In [ ]:
!git commit data/athletes.csv.dvc -m "Updated data to version 2"


In [ ]:
!git push